In [15]:
# Import dependencies
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


In [22]:

#  Import and read the flights_data.csv.
flights_df = pd.read_csv("flights_data_df.csv")
flights_df.head()


,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,Airline,Origin,OriginCityName,Dest,DestCityName,...,Cancelled,CancellationCode,CRSElapsedTime,ActualElapsedTime,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime
0,0,2021,1,2,6,Delta Air Lines Inc.,CVG,"Cincinnati, OH",BOS,"Boston, MA",...,0.0,NaN,129.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2021,1,3,7,Delta Air Lines Inc.,CVG,"Cincinnati, OH",BOS,"Boston, MA",...,0.0,NaN,129.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2021,1,4,1,Delta Air Lines Inc.,CVG,"Cincinnati, OH",BOS,"Boston, MA",...,0.0,NaN,129.0,121.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2021,1,7,4,Delta Air Lines Inc.,CVG,"Cincinnati, OH",BOS,"Boston, MA",...,0.0,NaN,117.0,121.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2021,1,8,5,Delta Air Lines Inc.,CVG,"Cincinnati, OH",BOS,"Boston, MA",...,0.0,NaN,117.0,135.0,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [26]:
# Creating 'Is_Delayed' column that will tell us if a flight is delayed or not. 
flights_df["Is_Delayed"] = None
flights_df.loc[flights_df["DepDelayMinutes"] == 0, "Is_Delayed"] = "Not_Delayed"
flights_df.loc[flights_df["DepDelayMinutes"] > 0, "Is_Delayed"] = "Delayed"


In [27]:
# cutting out a segment for the model
flights_df = flights_df[0:1000000]

In [28]:
# Filling NaN values
flights_df.fillna(0, inplace=True)

In [29]:
# Checking out dtypes
flights_df.dtypes[flights_df.dtypes == 'O']

Airline             object
Origin              object
OriginCityName      object
Dest                object
DestCityName        object
CancellationCode    object
Is_Delayed          object
dtype: object

In [30]:
#Drop unnecessary columns
#flights_df_segment = flights_data_segment.drop(['Origin', 'OriginCityName', 'DestCityName', 'Dest', 'Unnamed: 0', 'Cancelled', 'CarrierDelay', 'WeatherDelay', "NASDelay", 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'CancellationCode'], axis=1)
flights_data = flights_df.drop(['Year', 'DayofMonth', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'CRSElapsedTime', 'ActualElapsedTime', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Origin', 'OriginCityName', 'DestCityName', 'Unnamed: 0', 'Cancelled', 'FirstDepTime', 'CancellationCode'], axis=1)
flights_data

,Month,DayOfWeek,Airline,Dest,Is_Delayed
0,1,6,Delta Air Lines Inc.,BOS,Delayed
1,1,7,Delta Air Lines Inc.,BOS,Not_Delayed
2,1,1,Delta Air Lines Inc.,BOS,Delayed
3,1,4,Delta Air Lines Inc.,BOS,Not_Delayed
4,1,5,Delta Air Lines Inc.,BOS,Not_Delayed
...,...,...,...,...,...
999995,10,3,Delta Air Lines Inc.,IND,Not_Delayed
999996,10,4,Delta Air Lines Inc.,IND,Not_Delayed
999997,10,5,Delta Air Lines Inc.,IND,Not_Delayed
999998,10,6,Delta Air Lines Inc.,IND,Not_Delayed


In [31]:
df = pd.get_dummies(flights_data, columns=["Airline", "Dest"])

### Splitting our data into training and testing sets

In [32]:
# Create our features
X = df.drop(columns="Is_Delayed", axis=1) 


# Create our target
y = flights_df["Is_Delayed"]

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, test_size=0.3, stratify=y)
Counter(y_train)

Counter({'Delayed': 156946, 'Not_Delayed': 543054})

### Oversampling training data

In [36]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)


Counter({'Delayed': 543054, 'Not_Delayed': 543054})

### Scaling our data

In [40]:
from sklearn.preprocessing import StandardScaler

# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_resampled)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_resampled)
X_test_scaled = X_scaler.transform(X_test)

### Applying Logistic Regression model 

In [41]:

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_resampled)


LogisticRegression(random_state=1)

### Making predictions and Checking Accuracy

In [42]:

from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.5870025387730418

In [43]:

#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix

# Display the confusion matrix
y_pred = eec.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

C:\Users\Montse\anaconda3\envs\mlenv\lib\site-packages\sklearn\base.py:442: UserWarning: X does not have valid feature names, but EasyEnsembleClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([[ 67262,      0],
       [232738,      0]], dtype=int64)